In [1]:
from ReplayBuffer import ReplayBuffer
from DQN_Solver import DQN_Solver
from env import MPSPEnv
from DQN import DQN
import wandb
import numpy as np
import torch
import gym
import os
os.environ['WANDB_NOTEBOOK_NAME'] = 'main.ipynb'
wandb.login()

wandb: Currently logged in as: hojmax (rl-msps). Use `wandb login --relogin` to force relogin


True

In [2]:
config = {
    # Env
    'ROWS': 3,
    'COLUMNS': 3,
    'N_PORTS': 5,
    # Training
    'EPISODES': 1000,
    'LEARNING_RATE': 0.001,
    'MEM_SIZE': 10000,
    'BATCH_SIZE': 64,
    'GAMMA': 0.95,
    'EXPLORATION_MAX': 1.0,
    'EXPLORATION_DECAY': 0.99,
    'EXPLORATION_MIN': 0.001,
    'EVAL_EPISODES': 50,
    'MAX_EPISODE_STEPS': 200,
    # Model
    'HIDDEN_SIZE': 64,
    'N_LAYERS': 2,
}

In [3]:
env = gym.make('CartPole-v1')
# MPSPEnv(
#     config['ROWS'],
#     config['COLUMNS'],
#     config['N_PORTS']
# )
# We flatten the observation space
config['OBSERVATION_SPACE'] = env.observation_space.shape[0]
#  (
#     np.prod(env.observation_space[0].shape) +
#     np.prod(env.observation_space[1].shape)
# )
config['ACTION_SPACE'] = env.action_space.n


In [4]:
wandb.init(
    project="Q-learning",
    entity="rl-msps",
    name="cartpole",
    # name=f"{config['ROWS']}x{config['COLUMNS']}_{config['N_PORTS']}-ports",
    config=config,
    tags=["test"]
)

In [ ]:
ReplayBuffer = ReplayBuffer(
    mem_size=config['MEM_SIZE'],
    observation_space=config['OBSERVATION_SPACE'],
    batch_size=config['BATCH_SIZE']
)
DQN = DQN(
    input_size=config['OBSERVATION_SPACE'],
    output_size=config['ACTION_SPACE'],
    hidden_size=config['HIDDEN_SIZE'],
    n_layers=config['N_LAYERS'],
    learning_rate=config['LEARNING_RATE']
)
agent = DQN_Solver(
    ReplayBuffer=ReplayBuffer,
    DQN=DQN,
    batch_size=config['BATCH_SIZE'],
    exploration_max=config['EXPLORATION_MAX'],
    gamma=config['GAMMA'],
    exploration_decay=config['EXPLORATION_DECAY'],
    exploration_min=config['EXPLORATION_MIN']
)


In [ ]:
agent.train()

for i in range(1, config['EPISODES']):
    state, info = env.reset()
    # state = np.concatenate((state[0].flatten(), state[1].flatten()))
    sum_reward = 0
    sum_loss = 0
    iter = 0

    while iter < config['MAX_EPISODE_STEPS']:
        action, _ = agent.choose_action(state, np.ones(2, dtype=np.int8) , env) #info['mask']
        state_, reward, done, _, info = env.step(action)
        # state_ = np.concatenate((state_[0].flatten(), state_[1].flatten()))
        agent.memory.add(state, action, reward, state_, done)
        sum_loss += agent.learn()
        state = state_
        sum_reward += reward
        iter += 1

        if done:
            break

    wandb.log({
        "Sum Episode Reward": sum_reward,
        "Avg. Episode Loss": sum_loss / iter,
        "Exploration Rate": agent.exploration_rate
    })


In [ ]:
agent.eval()
sum_reward = 0

for i in range(1, config['EVAL_EPISODES']):
    state, info = env.reset()
    state = np.concatenate((state[0].flatten(), state[1].flatten()))

    while True:
        action, _ = agent.choose_action(state, info['mask'], env)
        state_, reward, done, info = env.step(action)
        state_ = np.concatenate((state_[0].flatten(), state_[1].flatten()))
        state = state_
        sum_reward += reward

        if done:
            break

wandb.summary['Avg. Eval Episode Reward'] = (
    sum_reward / config['EVAL_EPISODES']
)

In [ ]:
torch.save(DQN.state_dict(), os.path.join(wandb.run.dir, "dqn.pt"))
wandb.finish()